# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,18.22,82,20,5.81,PT,1706896951
1,1,valparaiso,-33.0393,-71.6273,38.41,35,8,6.69,CL,1706896804
2,2,port-aux-francais,-49.3500,70.2167,8.07,96,92,7.32,TF,1706896951
3,3,adamstown,-25.0660,-130.1015,24.63,84,33,4.95,PN,1706896951
4,4,afaahiti,-17.7500,-149.2833,27.93,72,85,4.48,PF,1706896952


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
cities_plot = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat',
    size='Humidity',
    scale=0.5,
    c='City',
    ylim=(85, -70),
    geo=True, 
    tiles='OSM',
    title='Cities and Their Relative Humidity'
)

# Display the map
cities_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
#Ideal Temp
ideal_cities_df = city_data_df[(city_data_df['Max Temp'] <= 16) & (city_data_df['Max Temp'] >= 9)]
#Ideal Humidity
ideal_cities_df = ideal_cities_df[(ideal_cities_df['Humidity'] >= 60) & (ideal_cities_df['Humidity'] < 90)]
#Ideal Cloudiness
ideal_cities_df = ideal_cities_df[ideal_cities_df['Cloudiness'] <= 40]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
105,105,calatayud,41.3535,-1.6432,11.80,72,10,0.46,ES,1706896976
128,128,blackmans bay,-43.0167,147.3167,10.07,78,17,2.86,AU,1706896981
156,156,la passe,45.5549,-0.8967,11.13,86,19,2.73,FR,1706896989
207,207,trapani,37.8333,12.6667,10.71,86,40,4.01,IT,1706897003
209,209,gasa,31.5000,34.4667,13.16,75,21,3.19,PS,1706897003
217,217,jbail,34.1211,35.6481,12.80,71,12,3.77,LB,1706897005
241,241,lompoc,34.6391,-120.4579,14.36,63,0,2.68,US,1706897012
242,242,christchurch,-43.5333,172.6333,9.97,87,7,0.45,NZ,1706896816
282,282,tympaki,35.0729,24.7685,10.12,68,25,1.34,GR,1706897022
285,285,zaragoza,41.6561,-0.8773,15.45,66,0,3.60,ES,1706896980


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.copy(deep=True)
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
105,calatayud,ES,41.3535,-1.6432,72,
128,blackmans bay,AU,-43.0167,147.3167,78,
156,la passe,FR,45.5549,-0.8967,86,
207,trapani,IT,37.8333,12.6667,86,
209,gasa,PS,31.5000,34.4667,75,
217,jbail,LB,34.1211,35.6481,71,
241,lompoc,US,34.6391,-120.4579,63,
242,christchurch,NZ,-43.5333,172.6333,87,
282,tympaki,GR,35.0729,24.7685,68,
285,zaragoza,ES,41.6561,-0.8773,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError) as e:
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
calatayud - nearest hotel: Arco San Miguel
blackmans bay - nearest hotel: Villa Howden
la passe - nearest hotel: L'Estuaire
trapani - nearest hotel: No hotel found
gasa - nearest hotel: مسجد الفتح المبين
jbail - nearest hotel: فندق كلف بوتيك
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
christchurch - nearest hotel: Ibis Hotel Christchurch
tympaki - nearest hotel: Sifogiannis
zaragoza - nearest hotel: Hotel Pilar Plaza
codrington - nearest hotel: No hotel found
camalu - nearest hotel: Margaritaville
segovia - nearest hotel: No hotel found
chowchilla - nearest hotel: Holiday Inn Express & Suites
saint-tropez - nearest hotel: No hotel found
arroyo de la luz - nearest hotel: Hotel Rural Villa Matilde
jinghong - nearest hotel: โรงแรมสืเบา


,City,Country,Lat,Lng,Humidity,Hotel Name
105,calatayud,ES,41.3535,-1.6432,72,Arco San Miguel
128,blackmans bay,AU,-43.0167,147.3167,78,Villa Howden
156,la passe,FR,45.5549,-0.8967,86,L'Estuaire
207,trapani,IT,37.8333,12.6667,86,No hotel found
209,gasa,PS,31.5000,34.4667,75,مسجد الفتح المبين
217,jbail,LB,34.1211,35.6481,71,فندق كلف بوتيك
241,lompoc,US,34.6391,-120.4579,63,Embassy Suites by Hilton Lompoc Central Coast
242,christchurch,NZ,-43.5333,172.6333,87,Ibis Hotel Christchurch
282,tympaki,GR,35.0729,24.7685,68,Sifogiannis
285,zaragoza,ES,41.6561,-0.8773,66,Hotel Pilar Plaza


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    x='Lng', 
    y='Lat',
    size='Humidity',
    scale=0.5,
    c='City',
    hover_cols=['Lng', 'Lat', 'City', 'Hotel Name', 'Country'],
    ylim=(60, -45),
    geo=True, 
    tiles='OSM',
    title='Cities with Targeted Weather'
)

# Display the map
hotel_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)